# 📅 Load Date table to Bronze, Silver and Gold 🥉🥈🥇

Used for date logic and to make it easier to do date-based reports.

In [1]:
CREATE OR REPLACE TEMPORARY VIEW DayNameReference AS
SELECT 1 AS DayOfWeek, 'Sunday' AS DayName
UNION 
SELECT 2 AS DayOfWeek, 'Monday' AS DayName
UNION 
SELECT 3 AS DayOfWeek, 'Tuesday' AS DayName
UNION 
SELECT 4 AS DayOfWeek, 'Wednesday' AS DayName
UNION 
SELECT 5 AS DayOfWeek, 'Thursday' AS DayName
UNION 
SELECT 6 AS DayOfWeek, 'Friday' AS DayName
UNION 
SELECT 7 AS DayOfWeek, 'Saturday' AS DayName

StatementMeta(, 663077f3-a02d-4019-9d78-b41db78a2fb1, 2, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [2]:
CREATE OR REPLACE TEMPORARY VIEW MonthNameReference AS
SELECT 1 AS MonthOfYear, 'January' AS MonthName
UNION 
SELECT 2 AS MonthOfYear, 'February' AS MonthName
UNION 
SELECT 3 AS MonthOfYear, 'March' AS MonthName
UNION 
SELECT 4 AS MonthOfYear, 'April' AS MonthName
UNION 
SELECT 5 AS MonthOfYear, 'May' AS MonthName
UNION 
SELECT 6 AS MonthOfYear, 'June' AS MonthName
UNION 
SELECT 7 AS MonthOfYear, 'July' AS MonthName
UNION
SELECT 8 AS MonthOfYear, 'August' AS MonthName
UNION 
SELECT 9 AS MonthOfYear, 'September' AS MonthName
UNION 
SELECT 10 AS MonthOfYear, 'October' AS MonthName
UNION 
SELECT 11 AS MonthOfYear, 'November' AS MonthName
UNION 
SELECT 12 AS MonthOfYear, 'December' AS MonthName

StatementMeta(, 663077f3-a02d-4019-9d78-b41db78a2fb1, 3, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
%%pyspark
from pyspark.sql import functions as F
start = "2000-01-01"
end = "2099-12-31"

'''
a = [(start,end)]
df = spark.createDataFrame(a, ["minDate", "maxDate"])
from pyspark.sql import functions as F
'''
df = spark.createDataFrame([(1,)])

df = df.withColumn(
    "date", 
    F.explode(F.expr("sequence(to_date('2000-01-01'), to_date('2029-12-31'), interval 1 day)"))
)

df = df.withColumn("DateKey",  F.date_format("date", "yyyyMMdd"))

df = df["DateKey","date"]

df.createOrReplaceTempView("datedim")

#df.show()
#Date;Day;DayInWeekNum;DayName;WeekNum;MonthNum;MonthName;Year

StatementMeta(, 663077f3-a02d-4019-9d78-b41db78a2fb1, 5, Finished, Available)

In [4]:
CREATE OR REPLACE TEMPORARY VIEW DateFlags AS
SELECT 
      date_format(Date, "yyyyMMdd") AS DateKey,
      timestamp(Date) AS Date,
      Day(Date) AS Day,
      CASE WHEN DayOfWeek(Date) IN (2, 3, 4, 5, 6, 7) THEN DayOfWeek(Date)-1
           WHEN DayOfWeek(Date) = 1 THEN 7 END AS DayInWeekNum, --(Databricks function indexed with Sunday = 1, Monday = 2, ..., Saturday = 7) 
      dnr.DayName AS DayName,
      weekofyear(Date) as WeekNum,
      Month(Date) AS MonthNum,
      mnr.MonthName AS MonthName,
      concat(mnr.MonthName, " ", Year(Date)) AS MonthAndYearName,
      date_format(Date, "yyyyMM") AS YearMonth,
      CONCAT('Q', Quarter(Date)) AS Quarter,
      CONCAT(Year(Date),'Q',Quarter(Date)) AS YearQuarter,
      Year(Date) AS Year,
      datediff(Date, current_date) AS DaysBetween,
      cast(months_between(date_format(Date, 'yyyy-MM'), date_format(current_date, 'yyyy-MM')) as int) AS MonthsBetween,
      case when current_date = Date then "Y" else "N" end AS TodayFlag,
      case when Year(current_date) = Year(Date) AND WeekOfYear(current_date) = WeekOfYear(Date) then "Y" else "N" end AS ThisWeekFlag,
      case when Year(current_date) = Year(Date) AND Month(current_date) = Month(Date) then "Y" else "N" end AS ThisMonthFlag,
      case when Year(current_date) = Year(Date) AND Month(current_date)>1 AND Month(current_date) = Month(Date)+1 then "Y" 
           when Year(current_date) = Year(Date)+1 AND Month(current_date)=1 AND Month(Date)=12 then "Y" 
           else "N" end AS LastMonthFlag,
      case when Year(current_date) = Year(Date) AND Quarter(current_date) = Quarter(Date) then "Y" 
           else "N" end AS ThisQuarterFlag,
      case when Year(current_date) = Year(Date) then "Y" ELSE "N" end AS ThisYearFlag,
      current_timestamp AS CreatedDateTime
FROM datedim

LEFT JOIN DayNameReference dnr ON dnr.DayOfWeek = DayOfWeek(Date)

LEFT JOIN MonthNameReference mnr ON mnr.MonthOfYear = Month(Date)

StatementMeta(, 663077f3-a02d-4019-9d78-b41db78a2fb1, 6, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [5]:
CREATE OR REPLACE TEMPORARY VIEW MonthsAlone AS
SELECT DISTINCT
      date_format(Date, "yyyyMM00") AS DateKey,
      cast(null as timestamp) AS Date,
      cast(null as int) AS Day,
      cast(null as int) AS DayInWeekNum,
      cast(null as string) AS DayName,
      cast(null as int) AS WeekNum,
      Month(Date) AS MonthNum,
      mnr.MonthName AS MonthName,
      concat(mnr.MonthName, " ", Year(Date)) AS MonthAndYearName,
      date_format(Date, "yyyyMM") AS YearMonth,
      CONCAT('Q', Quarter(Date)) AS Quarter,
      CONCAT(Year(Date),'Q',Quarter(Date)) AS YearQuarter,
      Year(Date) AS Year,
      cast(null as int) AS DaysBetween,
      cast(months_between(date_format(Date, 'yyyy-MM'), date_format(current_date, 'yyyy-MM')) as int) AS MonthsBetween,
      case when date_format(current_date(), "yyyy-MM") = date_format(Date, "yyyy-MM") then "Y" else "N" end AS TodayFlag,
      cast(null as string) AS ThisWeekFlag,
      case when Year(current_date) = Year(Date) AND Month(current_date) = Month(Date) then "Y" else "N" end AS ThisMonthFlag,
      case when Year(current_date) = Year(Date) AND Month(current_date)>1 AND Month(current_date) = Month(Date)+1 then "Y" 
           when Year(current_date) = Year(Date)+1 AND Month(current_date)=1 AND Month(Date)=12 then "Y" 
           else "N" end AS LastMonthFlag,
      case when Year(current_date) = Year(Date) AND Quarter(current_date) = Quarter(Date) then "Y" 
           else "N" end AS ThisQuarterFlag,
      case when Year(current_date) = Year(Date) then "Y" ELSE "N" end AS ThisYearFlag,
      current_timestamp AS CreatedDateTime
FROM datedim

LEFT JOIN DayNameReference dnr ON dnr.DayOfWeek = DayOfWeek(Date)

LEFT JOIN MonthNameReference mnr ON mnr.MonthOfYear = Month(Date)

StatementMeta(, 663077f3-a02d-4019-9d78-b41db78a2fb1, 7, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [6]:
CREATE OR REPLACE TEMPORARY VIEW DateView AS
SELECT * FROM DateFlags
UNION
SELECT * FROM MonthsAlone
ORDER BY DateKey

StatementMeta(, 663077f3-a02d-4019-9d78-b41db78a2fb1, 8, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [7]:
MERGE INTO Bronze_Airbnb.Date USING DateView ON Bronze_Airbnb.Date.DateKey = DateView.DateKey
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *

StatementMeta(, 663077f3-a02d-4019-9d78-b41db78a2fb1, 9, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>

In [8]:
MERGE INTO Silver_Airbnb.Date USING DateView ON Silver_Airbnb.Date.DateKey = DateView.DateKey
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *

StatementMeta(, 663077f3-a02d-4019-9d78-b41db78a2fb1, 10, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>

In [9]:
MERGE INTO Gold_Airbnb.DimDate USING DateView ON Gold_Airbnb.DimDate.DateKey = DateView.DateKey
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *

StatementMeta(, 663077f3-a02d-4019-9d78-b41db78a2fb1, 11, Finished, Available)